<a href="https://colab.research.google.com/github/raghavaravi/supplychainnetworkdesign/blob/main/SCM_275x_Graded_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Graded Assignment 1**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 25.7 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Ploting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi'), default value = 'km'

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit ('km' or 'mi')
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows on a map**

In [ ]:

# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


## **Data setup and preprocessing**

### **Nodes**

#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand

        self.active = True        # Initializing node as active


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply

        self.active = True      # Initializing node as active

In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, capacity, fixed_cost):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.capacity = capacity      # Warehouse's capacity
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost

        self.active = True            # Initializing node as active



#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/graded_assignment_1_facility_location/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,state,lat,lon,demand
0,c_01,Colorado Springs,Colorado,38.8674,-104.7605,400
1,c_02,Pittsburgh,Pennsylvania,40.4397,-79.9763,390
2,c_03,Charleston,South Carolina,32.8168,-79.9687,370
3,c_04,Rochester,New York,43.1680,-77.6162,210
4,c_05,Fresno,California,36.7830,-119.7939,250


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/graded_assignment_1_facility_location/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,state,lat,lon,supply
0,s_01,Concord,California,37.972200,-122.001600,5400
1,s_02,New Orleans,Louisiana,30.068700,-89.928800,4320
2,s_03,Boston,Massachusetts,42.318800,-71.085200,5220
3,s_04,Columbus,Ohio,39.986200,-82.985500,3780
4,s_05,Bismarck,North Dakota,46.825905,-100.778275,5680


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/graded_assignment_1_facility_location/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,state,lat,lon,capacity,fixed_cost
0,w_01,Hartford,Connecticut,41.7661,-72.6834,4200,46200
1,w_02,Bronx,New York,40.8501,-73.8662,5100,61200
2,w_03,Omaha,Nebraska,41.2627,-96.0529,5800,69600
3,w_04,Salt Lake City,Utah,40.7776,-111.9311,5900,59000
4,w_05,Orlando,Florida,28.4773,-81.3370,5800,69600


#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **suppliers}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    capacity = row['capacity'],     # Warehouse's capacity
                                    fixed_cost = row['fixed_cost']) # Warehouse's fixed cost

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

***❗Task 1: Extending the distance computation***

In the following code, we compute the distances for arcs connecting suppliers and warehouses, as well as arcs connecting warehouses and customers. We then compute and print the average arc distance in the network.

Modify the code to also calculate the distances between suppliers and customers. Make sure to store these values in the `distances` dictionary.


In [ ]:
# Creating a dictionary containing distances between suppliers and warehouses and warehouses and customers

distances = dict()
for s, supplier in suppliers.items():
  for w, warehouse in warehouses.items():
      distances[s, w] = compute_geodesic_distance(origin = supplier, destination = warehouse, unit = 'km')

for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

average_arc_distance = sum(distances.values()) / len(distances)
print ('The average arc distance before adding supplier-customer arcs is', round(average_arc_distance,1))

##### Task 1 : Your code here

average_arc_distance = sum(distances.values()) / len(distances)
print ('The average arc distance after adding supplier-customer arcsis', round(average_arc_distance,1))


The average arc distance before adding supplier-customer arcs is 1830.4
The average arc distance after adding supplier-customer arcsis 1830.4


#### Arc costs

***❗Task 2: Extending the unit transportation cost computation***

In the following code, we compute the unit transportation cost for arcs connecting suppliers and warehouses, as well as arcs connecting warehouses and customers. We then compute and print the average unit arc cost in the network.

Modify the code to also calculate the unit transportation cost between suppliers and customers. Consider that the cost per unit per kilometer between suppliers and customers is 0.35 (which is slightly higher than the cost between warehouses and customers (i.e., 0.30)). Make sure to store these values in the `unit_cost` dictionary.


In [ ]:
cost_unit_km_supplier_warehouse = 0.20 # Cost per unit per kilometer from supplier to warehouse
cost_unit_km_warehouse_customer = 0.30 # Cost per unit per kilometer from warehouse to customer


# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        unit_cost[s, w] = distances[s, w] * cost_unit_km_supplier_warehouse                    # Calculate unit cost as distance multiplied by cost per km (supplier to warehouse)

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)

average_arc_unit_cost = sum(unit_cost.values()) / len(unit_cost)
print ('The average unit arc cost before adding supplier-customer arcs is', round(average_arc_unit_cost,1))


##### Task 2 : Your code here

average_arc_unit_cost = sum(unit_cost.values()) / len(unit_cost)
print ('The average unit arc cost after adding supplier-customer arcs is', round(average_arc_unit_cost,1))


The average unit arc cost before adding supplier-customer arcs is 534.0
The average unit arc cost after adding supplier-customer arcs is 534.0


## **Optimization model**

### **Creating and solving the optimization model**

***❗Task 3: Extending the definition of decision variables***

In the following code, we formulate an optimization model with a flow decision variable defined on arcs connecting suppliers and warehouses, as well as arcs connecting  warehouses and customers. Modify the code to also define flow decision variables between suppliers and customers. Make sure to store these decision variables in the flow_vars dictionary.

***❗Task 4: Extending the objective function***

In the following code, we defined the total cost (i.e., the objective function) as the sum of the following cost components:
- Fixed cost of warehouses (i.e., fixed_cost_warehouse)
- Transportation cost between suppliers and warehouses (i.e., cost_supplier_warehouse)
- Transportation cost between warehouses and customers (i.e., cost_warehouse_customer)

Modify the code to:
- Compute the value of transportation cost between suppliers and customers (i.e., cost_supplier_customer)
- Include this value in the total cost


***❗Task 5: Extending the demand constraint***

In the following code, we defined demand constraints to ensure that the total incoming flow to each customer location matches the customer’s demand. To calculate the total incoming flow, we initially considered flows on arcs connecting warehouses to customers.


In our updated problem, we now include arcs connecting suppliers directly to customers. Adjust these constraints to account for all flows from both suppliers and warehouses.

***❗Task 6: Extending the supply constraint***

In the following code, we defined supply constraints to ensure that the total outgoing flow from each supplier location does not exceed the supplier's available supply. To calculate the total outgoing flow, we initially considered flows on arcs connecting suppliers to warehouses.


In our updated problem, we now include arcs connecting suppliers directly to customers. Adjust these constraints to account for all outgoing flow to both customers and warehouses.

In [ ]:
# Initializing the optimization model for the Facility Location Problem
model = grb.Model("Facility Location Problem")

# Creating decision variables

# Decision variable representing the flows between suppliers and warehouses, and warehouses and customers
flow_vars = dict()
for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        flow_vars[s, w] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_{0}_{1}".format(s, w))

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        flow_vars[w, c] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_{0}_{1}".format(w, c))

##### Task 3 : Your code here


# Binary decision variables to determine if a warehouse is opened
wh_location_vars = dict()
for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_vars{0}".format(w))

# Creating the objective function

# Fixed cost for opening warehouses
fixed_cost_warehouse = grb.quicksum(wh_location_vars[w] * warehouses[w].fixed_cost
                                       for w, warehouse in warehouses.items())

# Cost for flows from suppliers to warehouses
cost_supplier_warehouse = grb.quicksum(unit_cost[s, w] * flow_vars[s, w]
                                       for s, supplier in suppliers.items()
                                       for w, warehouse in warehouses.items())

# Cost for flows from warehouses to customers
cost_warehouse_customer = grb.quicksum(unit_cost[w, c] * flow_vars[w, c]
                                       for w, warehouse in warehouses.items()
                                       for c, customer in customers.items())



# Cost for flows from suppliers to customers

##### Task 4 : Your code here
## cost_supplier_customer =



# Total cost is the sum of flow costs and fixed warehouse costs
total_cost = cost_supplier_warehouse + cost_warehouse_customer + fixed_cost_warehouse

# Setting the objective to minimize total cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)


##### Task 5 : Your code here

# Adding demand constraints
for c, customer in customers.items():                                                          # Iterate over customers
    model.addConstr(grb.quicksum(flow_vars[w, c] for w, warehouse in warehouses.items()) == customer.demand)  # Flow into customer must meet demand


##### Task 6 : Your code here

# Adding supply constraints
for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    model.addConstr(grb.quicksum(flow_vars[s, w] for w, warehouse in warehouses.items()) <= supplier.supply)  # Flow out of supplier cannot exceed supply

# Adding flow preservation constraints
for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    model.addConstr(grb.quicksum(flow_vars[s, w] for s, supplier in suppliers.items()) ==       # Flow into warehouse must equal flow out
                    grb.quicksum(flow_vars[w, c] for c, customer in customers.items()))

# Adding warehouse capacity constraints (flow into warehouse must not exceed capacity if it is open)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[s, w] for s in suppliers) <= warehouse.capacity * wh_location_vars[w])

# Solving the optimization model
model.optimize()

# Output results
print('Fixed Cost Warehouses: ', fixed_cost_warehouse.getValue())
print('Number of Active Warehouses: ', grb.quicksum(wh_location_vars[w] for w in warehouses.keys()).getValue())

# Updating warehouse status based on optimization results
for w, warehouse in warehouses.items():
  if wh_location_vars[w].X == 1:  # If the warehouse is selected
    warehouse.active = True
  else:                           # If the warehouse is not selected
    warehouse.active = False

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 90 rows, 710 columns and 1470 nonzeros
Model fingerprint: 0xf867b615
Variable types: 700 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [3e+00, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+03]
Presolve time: 0.00s
Presolved: 90 rows, 710 columns, 1470 nonzeros
Variable types: 700 continuous, 10 integer (10 binary)
Found heuristic solution: objective 5786668.3688

Root relaxation: objective 5.410708e+06, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

## **Solution visualization and analysis**

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Plot the flows with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars, nodes = nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map
